In [2]:
import cv2
import numpy as np

In [ ]:
def is_solid_color_region(region, threshold=3.0):
    """region: (H, W, 3) の配列"""
    std = np.std(region.reshape(-1, 3), axis=0).mean()
    return std < threshold

def trim_padding_safely(img, scan_width=10, color_diff_th=10):
    h, w, _ = img.shape

    # 端から scan_width px の領域
    left_region   = img[:, :scan_width]
    right_region  = img[:, -scan_width:]
    top_region    = img[:scan_width, :]
    bottom_region = img[-scan_width:, :]

    # パディングかどうか判定
    trim_left   = is_solid_color_region(left_region)
    trim_right  = is_solid_color_region(right_region)
    trim_top    = is_solid_color_region(top_region)
    trim_bottom = is_solid_color_region(bottom_region)

    # 基準色（各端の平均）
    left_color  = np.mean(left_region.reshape(-1, 3), axis=0)
    right_color = np.mean(right_region.reshape(-1, 3), axis=0)
    top_color   = np.mean(top_region.reshape(-1, 3), axis=0)
    bottom_color= np.mean(bottom_region.reshape(-1, 3), axis=0)

    def color_diff(a, b):
        return np.linalg.norm(a - b)

    # 境界を探す
    left = 0
    if trim_left:
        for x in range(w):
            col = img[:, x, :]
            c = np.mean(col, axis=0)
            if color_diff(c, left_color) > color_diff_th:
                left = x
                break

    right = w - 1
    if trim_right:
        for x in range(w-1, -1, -1):
            col = img[:, x, :]
            c = np.mean(col, axis=0)
            if color_diff(c, right_color) > color_diff_th:
                right = x
                break

    top = 0
    if trim_top:
        for y in range(h):
            row = img[y, :, :]
            c = np.mean(row, axis=0)
            if color_diff(c, top_color) > color_diff_th:
                top = y
                break

    bottom = h - 1
    if trim_bottom:
        for y in range(h-1, -1, -1):
            row = img[y, :, :]
            c = np.mean(row, axis=0)
            if color_diff(c, bottom_color) > color_diff_th:
                bottom = y
                break

    # トリミング
    return img[top:bottom+1, left:right+1]


In [ ]:
import os

for filename in os.listdir("images/raw"):
    img = cv2.imread(f"images/raw/{filename}")
    h, w, c = img.shape

    # 左右分割
    left_img  = img[:, :w//2]
    right_img = img[:, w//2:]

    # それぞれのパディング削除
    left_trimmed = trim_padding_safely(left_img)
    right_trimmed = trim_padding_safely(right_img)

    # 保存
    # cv2.imwrite(f"images/trimmed/{filename[:-5]}_left.png", left_trimmed)
    # cv2.imwrite(f"images/trimmed/{filename[:-5]}_right.png", right_trimmed)